# quantization

This is an implementation of the model using dynamic quantization. This model is derived from `model_1.py`, *not* `model_2.py` (the former is the baseline model, the latter implements mixed precision training; we aren't doing _both_ quantization and mixed precision training here).

In [1]:
import transformers

cfg = transformers.PretrainedConfig.get_config_dict("bert-base-uncased")[0]  # tuple?
cfg["output_hidden_states"] = True
cfg = transformers.BertConfig.from_dict(cfg)
bert = transformers.BertModel.from_pretrained("bert-base-uncased", config=cfg)

In [3]:
import torch.quantization
qbert = torch.quantization.quantize_dynamic(
    bert, {torch.nn.Linear}, dtype=torch.qint8
)

Notice the new `DynamicQuantizedLinear` layers in the model `repr`:

In [4]:
qbert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): DynamicQuantizedLinear(in_features=768, out_features=768, scale=1.0, zero_point=0)
            (key): DynamicQuantizedLinear(in_features=768, out_features=768, scale=1.0, zero_point=0)
            (value): DynamicQuantizedLinear(in_features=768, out_features=768, scale=1.0, zero_point=0)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): DynamicQuantizedLinear(in_features=768, out_features=768, scale=1.0, zero_point=0)
            (LayerNorm

The run command is:

```bash
spell run \
  --machine-type t4 \
  --github-url https://github.com/spellml/tweet-sentiment-extraction.git \
  --pip transformers==3.5.1 --pip tokenizers --pip kaggle --pip tensorboard \
  --env KAGGLE_USERNAME=YOUR_USERNAME \
  --env KAGGLE_KEY=YOUR_KEY \
  --tensorboard-dir /spell/tensorboards/model_3 \
  "chmod +x /spell/scripts/download_data.sh; /spell/scripts/download_data.sh; python /spell/models/model_2.py"
```

You can't even load a dynamically quantized model on a GPU, it turns out. You have to do so in a CPU context. The process seems to be to attach quantization to the model on the _target_ machine.

In [9]:
# !mkdir ../servers

In [1]:
%%writefile ../servers/eval.py
import os
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import transformers
import tokenizers
from transformers import AdamW, AutoTokenizer, get_linear_schedule_with_warmup
import time

train = pd.read_csv("/mnt/tweet-sentiment-extraction/train.csv")

def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    # yes this really happens lol, try idx 314
    if pd.isnull(tweet) or pd.isnull(selected_text) or len(tweet) == 0 or len(selected_text) == 0:
        raise ValueError("text or selected_text is nan.")
    
    # get indicial boundaries of substring.
    target_char_idx_start = tweet.index(selected_text)
    target_char_idx_end = target_char_idx_start + len(selected_text)

    # build the character attention mask (used to build the token attention mask)
    char_target_mask = (
        [0] * target_char_idx_start +
        [1] * (target_char_idx_end - target_char_idx_start) +
        [0] * (len(tweet) - target_char_idx_end)
    )
    
    # tokenize
    # `ids` is the token values, `offsets` are the position tuples for the tokes in the str
    tokens_obj = tokenizer.encode(tweet)
    token_ids, token_offsets = tokens_obj.ids, tokens_obj.offsets
    
    # this is the clever bit. recall that the task is to find the subsequence in the sequence
    # exemplifying the given sentiment. to do this we reformulate the input sequence as a
    # question-answer pair, where the sentiment (as a single word) is the question and the
    # sequence as a whole is the answer.
    # 
    # this allows us to use version of the BERT model pretrained on the well-formed and
    # well-studied question-answering task as a surrogate for this task.
    sentiment_id_map = {
        'positive': 3893,
        'negative': 4997,
        'neutral': 8699
    }
    # 101 is [CLS] and 102 is [SEP]. BERT expects Q/A input to be in the form
    # [CLS] [...] [SEP] [...] [SEP]. Cf.
    # https://huggingface.co/transformers/glossary.html#token-type-ids
    # NOTE: the [-1:1] is the excise the start-of-seq and end-of-seq in the tokens
    input_ids = [101] + [sentiment_id_map[sentiment]] + [102] + token_ids[1:-1] + [102]
    
    # BERT expects Q/A pairs to come with a binary mask splitting the pair types
    # NOTE: the mafs excludes start-of-seq and end-of-seq but includes the new end-of-seq
    token_type_ids = [0, 0, 0] + [1] * (len(token_ids) - 2 + 1)

    # pad to max_len and create a corresponding attention mask
    pad_len = max_len - len(input_ids)
    attention_mask = [1] * len(input_ids) + [0] * pad_len
    input_ids = input_ids + [0] * pad_len
    token_type_ids = token_type_ids + [0] * pad_len
    
    # get the index of the first and last token of the target, this is what the model will try
    # to predict! see the notes on the head layer in forward for more info.
    # we add 3 because the first thee elements of the mask are always [CLS] $SENTIMENT [CLS]
    # and always get an attention vector [1 1 1].
    ufunc = lambda first, _: first >= target_char_idx_start and first < target_char_idx_end
    y_pred_mask = [ufunc(*offset) for offset in token_offsets]
    try:
        y_first = 3 + y_pred_mask.index(True)
        y_last = 3 + len(y_pred_mask) - y_pred_mask[::-1].index(True) - 1
    except ValueError:
        # some of the labels are noisy, and the first character in the label does not actually
        # correspond with the first character of any token (e.g. the label is a part-of-a-word
        # instead of a word). I'm going to venture the opinion here that these records 
        # constitute data noise (because, I mean, they are) and should be removed in
        # pre-processing
        raise ValueError(
            f"Found bad selected_text value '{selected_text}' for tweet '{tweet}'."
            f"Make sure to get rid of these in a pre-processing pass."
        )
    
    # convert to torch tensors
    t = lambda seq: torch.tensor(seq, dtype=torch.long)
    input_ids, token_type_ids, attention_mask, y_first, y_last =\
        t(input_ids), t(token_type_ids), t(attention_mask), t(y_first), t(y_last)
    
    # output
    # Unfortunately the PyTorch dataloader relies on pickle, and TIL namedtuples do not play nice
    # with pickle!
    # Record = namedtuple('record', 'input_ids token_type_ids attention_mask y_first y_last')
    # return Record(input_ids, token_type_ids, attention_mask, y_first, y_last)
    return {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask,
        "y_first": y_first,
        "y_last": y_last
    }

class TwitterSentimentExtractionDataset:
    def __init__(self, df):
        self.df = df
        self.tokenizer = tokenizers.BertWordPieceTokenizer(
            f"/mnt/bert-base-uncased/vocab.txt", lowercase=True
        )
        self.max_len = 128
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, item):
        return process_data(
            self.df.text[item],
            self.df.selected_text[item], 
            self.df.sentiment[item],
            self.tokenizer,
            self.max_len
        )

# preprocessing pass; see comments in the previous code cell on why this is necessary
X_train_preprocessing_pass = TwitterSentimentExtractionDataset(train)

bad_idxs, good_idxs, y_firsts, y_lasts = [], [], [], []
for i in range(len(train)):
    try:
        x = X_train_preprocessing_pass[i]
        y_firsts.append(x['y_first'])
        y_lasts.append(x['y_last'])
        good_idxs.append(i)
    except ValueError:
        print(f"Found bad record at idx {i}.")
        y_firsts.append(None)
        y_lasts.append(None)
        bad_idxs.append(i)

del X_train_preprocessing_pass
train_orig = train
X_train_df = train.iloc[good_idxs].reset_index(drop=True)

class TwitterSentimentExtractionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        # configuring the model to output hidden states
        cfg = transformers.PretrainedConfig.get_config_dict("bert-base-uncased")[0]  # tuple?
        cfg["output_hidden_states"] = True
        cfg = transformers.BertConfig.from_dict(cfg)
        
        bert = transformers.BertModel.from_pretrained("bert-base-uncased", config=cfg)
        self.bert = bert
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
        # self.out = nn.LogSoftmax(dim=-2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # ignore the output from the model head, we'll instead we'll construct our own attention
        # head connected to the last two layers of hidden weights.
        # that's 512x762x2=780288 connections.
        _, _, out = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        # the new head uses a linear layer with two output nodes.
        # the first node learns sequence start.
        # the second node learns sequence end.
        logits = self.l0(out)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        
        return start_logits, end_logits
        
        # TODO: embed softmax directly into the model arch
        # y_start, y_end = self.out(start_logits), self.out(end_logits)
        # return y_start, y_end

# constants
batch_size = 64
if torch.cuda.device_count() >= 1:
    device = torch.device("cuda")
    is_cpu_run = False
else:
    device = torch.device("cpu")
    is_cpu_run = True

# dataset and dataloader
dataset = TwitterSentimentExtractionDataset(X_train_df)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    num_workers=1
)

# eval func for one epoch of evaluation
def eval_fn(dataloader, model, device):
    model.eval()
    
    fn = lambda field: records[field].to(device, dtype=torch.long)
    for idx, records in enumerate(dataloader):
        # move the record to GPU
        input_ids = fn("input_ids")
        token_type_ids = fn("token_type_ids")
        attention_mask = fn("attention_mask")
        y_first = fn("y_first")
        y_last = fn("y_last")
        
        y_pred_start_logits, y_pred_end_logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        y_pred_starts = torch.softmax(y_pred_start_logits, dim=1).cpu().detach().numpy()
        y_pred_ends = torch.softmax(y_pred_end_logits, dim=1).cpu().detach().numpy()
        return y_pred_starts, y_pred_ends


def main():
    checkpoints_dir = "/spell/checkpoints"
    model = TwitterSentimentExtractionModel()
    
    if is_cpu_run:
        weights = torch.load(f"{checkpoints_dir}/model_5.pth", map_location=torch.device('cpu'))
    else:
        weights = torch.load(f"{checkpoints_dir}/model_5.pth")

    model.load_state_dict(weights)
    model.to(device)
    model.eval()
    
    print(f"Evaluating the model...")
    start_time = time.time()
    eval_fn(dataloader, model, device)
    print(f"Evaluation done in {str(time.time() - start_time)} seconds.")

if __name__ == "__main__":
    main()

Overwriting ../servers/eval.py


```python
spell run \
  --machine-type t4 \
  --github-url https://github.com/spellml/tweet-sentiment-extraction.git \
  --pip transformers==3.5.1 --pip tokenizers --pip kaggle \
  --env KAGGLE_USERNAME=YOUR_USERNAME \
  --env KAGGLE_KEY=YOUR_KEY \
  --mount runs/414/checkpoints/model_5.pth:/spell/checkpoints/model_5.pth \
  "chmod +x /spell/scripts/download_data.sh; /spell/scripts/download_data.sh; python /spell/servers/eval.py"
```

In [26]:
%%writefile ../servers/eval_quantized.py
import os
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import transformers
import tokenizers
from transformers import AdamW, AutoTokenizer, get_linear_schedule_with_warmup
import time

train = pd.read_csv("/mnt/tweet-sentiment-extraction/train.csv")

def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    # yes this really happens lol, try idx 314
    if pd.isnull(tweet) or pd.isnull(selected_text) or len(tweet) == 0 or len(selected_text) == 0:
        raise ValueError("text or selected_text is nan.")
    
    # get indicial boundaries of substring.
    target_char_idx_start = tweet.index(selected_text)
    target_char_idx_end = target_char_idx_start + len(selected_text)

    # build the character attention mask (used to build the token attention mask)
    char_target_mask = (
        [0] * target_char_idx_start +
        [1] * (target_char_idx_end - target_char_idx_start) +
        [0] * (len(tweet) - target_char_idx_end)
    )
    
    # tokenize
    # `ids` is the token values, `offsets` are the position tuples for the tokes in the str
    tokens_obj = tokenizer.encode(tweet)
    token_ids, token_offsets = tokens_obj.ids, tokens_obj.offsets
    
    # this is the clever bit. recall that the task is to find the subsequence in the sequence
    # exemplifying the given sentiment. to do this we reformulate the input sequence as a
    # question-answer pair, where the sentiment (as a single word) is the question and the
    # sequence as a whole is the answer.
    # 
    # this allows us to use version of the BERT model pretrained on the well-formed and
    # well-studied question-answering task as a surrogate for this task.
    sentiment_id_map = {
        'positive': 3893,
        'negative': 4997,
        'neutral': 8699
    }
    # 101 is [CLS] and 102 is [SEP]. BERT expects Q/A input to be in the form
    # [CLS] [...] [SEP] [...] [SEP]. Cf.
    # https://huggingface.co/transformers/glossary.html#token-type-ids
    # NOTE: the [-1:1] is the excise the start-of-seq and end-of-seq in the tokens
    input_ids = [101] + [sentiment_id_map[sentiment]] + [102] + token_ids[1:-1] + [102]
    
    # BERT expects Q/A pairs to come with a binary mask splitting the pair types
    # NOTE: the mafs excludes start-of-seq and end-of-seq but includes the new end-of-seq
    token_type_ids = [0, 0, 0] + [1] * (len(token_ids) - 2 + 1)

    # pad to max_len and create a corresponding attention mask
    pad_len = max_len - len(input_ids)
    attention_mask = [1] * len(input_ids) + [0] * pad_len
    input_ids = input_ids + [0] * pad_len
    token_type_ids = token_type_ids + [0] * pad_len
    
    # get the index of the first and last token of the target, this is what the model will try
    # to predict! see the notes on the head layer in forward for more info.
    # we add 3 because the first thee elements of the mask are always [CLS] $SENTIMENT [CLS]
    # and always get an attention vector [1 1 1].
    ufunc = lambda first, _: first >= target_char_idx_start and first < target_char_idx_end
    y_pred_mask = [ufunc(*offset) for offset in token_offsets]
    try:
        y_first = 3 + y_pred_mask.index(True)
        y_last = 3 + len(y_pred_mask) - y_pred_mask[::-1].index(True) - 1
    except ValueError:
        # some of the labels are noisy, and the first character in the label does not actually
        # correspond with the first character of any token (e.g. the label is a part-of-a-word
        # instead of a word). I'm going to venture the opinion here that these records 
        # constitute data noise (because, I mean, they are) and should be removed in
        # pre-processing
        raise ValueError(
            f"Found bad selected_text value '{selected_text}' for tweet '{tweet}'."
            f"Make sure to get rid of these in a pre-processing pass."
        )
    
    # convert to torch tensors
    t = lambda seq: torch.tensor(seq, dtype=torch.long)
    input_ids, token_type_ids, attention_mask, y_first, y_last =\
        t(input_ids), t(token_type_ids), t(attention_mask), t(y_first), t(y_last)
    
    # output
    # Unfortunately the PyTorch dataloader relies on pickle, and TIL namedtuples do not play nice
    # with pickle!
    # Record = namedtuple('record', 'input_ids token_type_ids attention_mask y_first y_last')
    # return Record(input_ids, token_type_ids, attention_mask, y_first, y_last)
    return {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask,
        "y_first": y_first,
        "y_last": y_last
    }

class TwitterSentimentExtractionDataset:
    def __init__(self, df):
        self.df = df
        self.tokenizer = tokenizers.BertWordPieceTokenizer(
            f"/mnt/bert-base-uncased/vocab.txt", lowercase=True
        )
        self.max_len = 128
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, item):
        return process_data(
            self.df.text[item],
            self.df.selected_text[item], 
            self.df.sentiment[item],
            self.tokenizer,
            self.max_len
        )

# preprocessing pass; see comments in the previous code cell on why this is necessary
X_train_preprocessing_pass = TwitterSentimentExtractionDataset(train)

bad_idxs, good_idxs, y_firsts, y_lasts = [], [], [], []
for i in range(len(train)):
    try:
        x = X_train_preprocessing_pass[i]
        y_firsts.append(x['y_first'])
        y_lasts.append(x['y_last'])
        good_idxs.append(i)
    except ValueError:
        print(f"Found bad record at idx {i}.")
        y_firsts.append(None)
        y_lasts.append(None)
        bad_idxs.append(i)

del X_train_preprocessing_pass
train_orig = train
X_train_df = train.iloc[good_idxs].reset_index(drop=True)

class TwitterSentimentExtractionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        # configuring the model to output hidden states
        cfg = transformers.PretrainedConfig.get_config_dict("bert-base-uncased")[0]  # tuple?
        cfg["output_hidden_states"] = True
        cfg = transformers.BertConfig.from_dict(cfg)
        
        bert = transformers.BertModel.from_pretrained("bert-base-uncased", config=cfg)
        self.bert = bert
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
        # self.out = nn.LogSoftmax(dim=-2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # ignore the output from the model head, we'll instead we'll construct our own attention
        # head connected to the last two layers of hidden weights.
        # that's 512x762x2=780288 connections.
        _, _, out = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        # the new head uses a linear layer with two output nodes.
        # the first node learns sequence start.
        # the second node learns sequence end.
        logits = self.l0(out)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        
        return start_logits, end_logits
        
        # TODO: embed softmax directly into the model arch
        # y_start, y_end = self.out(start_logits), self.out(end_logits)
        # return y_start, y_end

# constants
batch_size = 64
device = torch.device("cpu")

# dataset and dataloader
dataset = TwitterSentimentExtractionDataset(X_train_df)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    num_workers=1
)

# eval func for one epoch of evaluation
def eval_fn(dataloader, model, device):
    model.eval()
    
    fn = lambda field: records[field].to(device, dtype=torch.long)
    for idx, records in enumerate(dataloader):
        # move the record to GPU
        input_ids = fn("input_ids")
        token_type_ids = fn("token_type_ids")
        attention_mask = fn("attention_mask")
        y_first = fn("y_first")
        y_last = fn("y_last")
        
        y_pred_start_logits, y_pred_end_logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        y_pred_starts = torch.softmax(y_pred_start_logits, dim=1).cpu().detach().numpy()
        y_pred_ends = torch.softmax(y_pred_end_logits, dim=1).cpu().detach().numpy()
        return y_pred_starts, y_pred_ends


def main():
    checkpoints_dir = "/spell/checkpoints"
    model = TwitterSentimentExtractionModel()
    model.load_state_dict(torch.load(f"{checkpoints_dir}/model_5.pth", map_location=torch.device('cpu')))
    qmodel = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
    qmodel.to(device)
    qmodel.eval()
    
    print(f"Evaluating the model...")
    start_time = time.time()
    eval_fn(dataloader, qmodel, device)
    print(f"Evaluation done in {str(time.time() - start_time)} seconds.")

if __name__ == "__main__":
    main()

Overwriting ../servers/eval_quantized.py


Localizing the data for development purposes.

In [3]:
# !pip install kaggle

In [47]:
# !pip install transformers==3.5.1

     |████████████████████████████████| 1.3 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 31.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 64.8 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.0.0
    Uninstalling transformers-4.0.0:
      Successfully uninstalled transformers-4.0.0
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## Generating an example model server input

In this section I go about generating an example input for the model server.

In [4]:
%%bash
export KAGGLE_USERNAME=residentmario
export KAGGLE_KEY=5ae93199f0523afe441e922fcdc75ee4
chmod +x /spell/scripts/download_data.sh; /spell/scripts/download_data.sh


Archive:  tweet-sentiment-extraction.zip
  inflating: /mnt/tweet-sentiment-extraction/sample_submission.csv  
  inflating: /mnt/tweet-sentiment-extraction/test.csv  
  inflating: /mnt/tweet-sentiment-extraction/train.csv  

Archive:  bert-base-uncased.zip
  inflating: /mnt/bert-base-uncased/config.json  
  inflating: /mnt/bert-base-uncased/pytorch_model.bin  
  inflating: /mnt/bert-base-uncased/vocab.txt  


100%|██████████| 1.39M/1.39M [00:00<00:00, 16.8MB/s]
100%|██████████| 389M/389M [00:06<00:00, 59.9MB/s] 


In [1]:
import os
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler
import transformers
import tokenizers
from transformers import AdamW, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter

train = pd.read_csv("/mnt/tweet-sentiment-extraction/train.csv")

def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    # yes this really happens lol, try idx 314
    if pd.isnull(tweet) or pd.isnull(selected_text) or len(tweet) == 0 or len(selected_text) == 0:
        raise ValueError("text or selected_text is nan.")
    
    # get indicial boundaries of substring.
    target_char_idx_start = tweet.index(selected_text)
    target_char_idx_end = target_char_idx_start + len(selected_text)

    # build the character attention mask (used to build the token attention mask)
    char_target_mask = (
        [0] * target_char_idx_start +
        [1] * (target_char_idx_end - target_char_idx_start) +
        [0] * (len(tweet) - target_char_idx_end)
    )
    
    # tokenize
    # `ids` is the token values, `offsets` are the position tuples for the tokes in the str
    tokens_obj = tokenizer.encode(tweet)
    token_ids, token_offsets = tokens_obj.ids, tokens_obj.offsets
    
    # this is the clever bit. recall that the task is to find the subsequence in the sequence
    # exemplifying the given sentiment. to do this we reformulate the input sequence as a
    # question-answer pair, where the sentiment (as a single word) is the question and the
    # sequence as a whole is the answer.
    # 
    # this allows us to use version of the BERT model pretrained on the well-formed and
    # well-studied question-answering task as a surrogate for this task.
    sentiment_id_map = {
        'positive': 3893,
        'negative': 4997,
        'neutral': 8699
    }
    # 101 is [CLS] and 102 is [SEP]. BERT expects Q/A input to be in the form
    # [CLS] [...] [SEP] [...] [SEP]. Cf.
    # https://huggingface.co/transformers/glossary.html#token-type-ids
    # NOTE: the [-1:1] is the excise the start-of-seq and end-of-seq in the tokens
    input_ids = [101] + [sentiment_id_map[sentiment]] + [102] + token_ids[1:-1] + [102]
    
    # BERT expects Q/A pairs to come with a binary mask splitting the pair types
    # NOTE: the mafs excludes start-of-seq and end-of-seq but includes the new end-of-seq
    token_type_ids = [0, 0, 0] + [1] * (len(token_ids) - 2 + 1)

    # pad to max_len and create a corresponding attention mask
    pad_len = max_len - len(input_ids)
    attention_mask = [1] * len(input_ids) + [0] * pad_len
    input_ids = input_ids + [0] * pad_len
    token_type_ids = token_type_ids + [0] * pad_len
    
    # get the index of the first and last token of the target, this is what the model will try
    # to predict! see the notes on the head layer in forward for more info.
    # we add 3 because the first thee elements of the mask are always [CLS] $SENTIMENT [CLS]
    # and always get an attention vector [1 1 1].
    ufunc = lambda first, _: first >= target_char_idx_start and first < target_char_idx_end
    y_pred_mask = [ufunc(*offset) for offset in token_offsets]
    try:
        y_first = 3 + y_pred_mask.index(True)
        y_last = 3 + len(y_pred_mask) - y_pred_mask[::-1].index(True) - 1
    except ValueError:
        # some of the labels are noisy, and the first character in the label does not actually
        # correspond with the first character of any token (e.g. the label is a part-of-a-word
        # instead of a word). I'm going to venture the opinion here that these records 
        # constitute data noise (because, I mean, they are) and should be removed in
        # pre-processing
        raise ValueError(
            f"Found bad selected_text value '{selected_text}' for tweet '{tweet}'."
            f"Make sure to get rid of these in a pre-processing pass."
        )
    
    # convert to torch tensors
    t = lambda seq: torch.tensor(seq, dtype=torch.long)
    input_ids, token_type_ids, attention_mask, y_first, y_last =\
        t(input_ids), t(token_type_ids), t(attention_mask), t(y_first), t(y_last)
    
    # output
    # Unfortunately the PyTorch dataloader relies on pickle, and TIL namedtuples do not play nice
    # with pickle!
    # Record = namedtuple('record', 'input_ids token_type_ids attention_mask y_first y_last')
    # return Record(input_ids, token_type_ids, attention_mask, y_first, y_last)
    return {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask,
        "y_first": y_first,
        "y_last": y_last
    }

class TwitterSentimentExtractionDataset:
    def __init__(self, df):
        self.df = df
        self.tokenizer = tokenizers.BertWordPieceTokenizer(
            f"/mnt/bert-base-uncased/vocab.txt", lowercase=True
        )
        self.max_len = 128
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, item):
        return process_data(
            self.df.text[item],
            self.df.selected_text[item], 
            self.df.sentiment[item],
            self.tokenizer,
            self.max_len
        )

In [2]:
ds = TwitterSentimentExtractionDataset(train)

In [3]:
ex = next(iter(ds))
ex

{'input_ids': tensor([ 101, 8699,  102, 1045, 1036, 1040, 2031, 5838, 1010, 2065, 1045, 2020,
         2183,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'token_type_ids': tensor([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0

In [4]:
ex_fmt = dict()
ex_fmt['input_ids'] = ex['input_ids'].tolist()
ex_fmt['token_type_ids'] = ex['token_type_ids'].tolist()
ex_fmt['attention_mask'] = ex['attention_mask'].tolist()
ex_fmt['y_first'] = ex['y_first'].item()
ex_fmt['y_last'] = ex['y_last'].item()

In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=0, compact=True)
pp.pprint(ex_fmt)

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0],
'input_ids': [101, 8699, 102, 1045, 1036, 1040, 2031, 5838, 1010, 2065, 1045,
             2020, 2183, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            

In [6]:
import json
with open("../servers/example_payload.json", "w") as fp:
    json.dump(ex_fmt, fp)

In [7]:
dataloader = torch.utils.data.DataLoader(
    ds,
    batch_size=1,
    num_workers=1
)

In [8]:
next(iter(dataloader))

{'input_ids': tensor([[ 101, 8699,  102, 1045, 1036, 1040, 2031, 5838, 1010, 2065, 1045, 2020,
          2183,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [9]:
input_ids = torch.tensor([ex_fmt['input_ids']])
token_type_ids = torch.tensor([ex_fmt['token_type_ids']])
attention_mask = torch.tensor([ex_fmt['attention_mask']])

In [10]:
class TwitterSentimentExtractionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        # configuring the model to output hidden states
        cfg = transformers.PretrainedConfig.get_config_dict("bert-base-uncased")[0]  # tuple?
        cfg["output_hidden_states"] = True
        cfg = transformers.BertConfig.from_dict(cfg)
        
        bert = transformers.BertModel.from_pretrained("bert-base-uncased", config=cfg)
        self.bert = bert
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
        # self.out = nn.LogSoftmax(dim=-2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # ignore the output from the model head, we'll instead we'll construct our own attention
        # head connected to the last two layers of hidden weights.
        # that's 512x762x2=780288 connections.
        _, _, out = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        # the new head uses a linear layer with two output nodes.
        # the first node learns sequence start.
        # the second node learns sequence end.
        logits = self.l0(out)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        
        return start_logits, end_logits
        
        # TODO: embed softmax directly into the model arch
        # y_start, y_end = self.out(start_logits), self.out(end_logits)
        # return y_start, y_end

In [11]:
model = TwitterSentimentExtractionModel()

In [12]:
model(
    input_ids=input_ids,
    token_type_ids=token_type_ids,
    attention_mask=attention_mask
)

(tensor([[ 0.3959, -0.1834,  0.1791,  0.6216,  0.0544, -0.3758, -0.5169,  0.4187,
          -0.2306, -0.4681,  0.4183,  0.2476,  0.2307, -0.0617,  0.5199,  0.8549,
           0.2622,  1.0879,  0.8102,  1.0117,  1.0577,  1.0207,  1.1073,  1.1544,
           1.0915,  0.8919,  0.7396,  0.3420,  0.8595,  0.7967,  0.8687,  1.0728,
           0.7715,  1.0828,  1.0236,  1.1534,  0.8812,  0.8963,  0.8666,  0.9138,
           0.2778,  0.7737,  0.7584,  1.0188,  1.1032,  1.0181,  1.1179,  1.2806,
           1.0602,  0.9708,  1.1679,  0.9774,  1.1420,  0.6806,  0.7948,  0.8754,
           1.0770,  0.7149,  0.9675,  0.7816,  0.8900,  1.2835,  1.1273,  1.2192,
          -0.4279,  0.9868,  1.0101,  1.1320,  1.0774,  1.4328,  0.6311,  0.9705,
           1.3023,  1.1205,  1.0002,  1.1522,  1.3430,  0.9718,  0.7316,  0.9448,
           0.9305,  1.0479,  0.9860,  0.9676,  0.8448,  0.9865,  1.0748,  0.6102,
           1.0593,  0.9066,  0.7934,  0.9831,  0.9595,  0.0537,  0.4025,  0.8291,
           0.448

In [13]:
payload = {'attention_mask': 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0],
'input_ids':
[101, 8699, 102, 1045, 1036, 1040, 2031, 5838, 1010, 2065, 1045,
 2020, 2183, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0],
'token_type_ids':
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0],
'y_first': 4,
'y_last': 13}
input_ids = torch.tensor([payload['input_ids']])
token_type_ids = torch.tensor([payload['token_type_ids']])
attention_mask = torch.tensor([payload['attention_mask']])
start_logits, end_logits = model(
input_ids=input_ids,
attention_mask=attention_mask,
token_type_ids=token_type_ids
)

In [21]:
t = torch.tensor([0])
t.to('cuda')

tensor([0], device='cuda:0')

In [22]:
torch.load

<function torch.serialization.load(f, map_location=None, pickle_module=<module 'pickle' from '/usr/lib/python3.7/pickle.py'>, **pickle_load_args)>